In [15]:
import pandas as pd
from omop_etl.datastore import DataStore
from omop_etl.stage import STAGE

In [2]:
omop = DataStore('../config.yml')

In [3]:
stage_t = omop.list_tables(in_schema=['stage'])
stage_t['Count'] = stage_t.Table.apply(lambda t: omop.row_count(t, schema='stage'))

In [27]:
stage_t_dict = stage_t[['Table','Count']].to_dict()
stage_t_dict

{'Table': {0: 'PERSON',
  1: 'DEATH',
  2: 'CONDITION',
  3: 'PROCEDURE_ICD',
  4: 'PROCEDURE_CPT',
  5: 'DRUG_ORDER',
  6: 'DRUG_ADMIN',
  7: 'MEASUREMENT_BP',
  8: 'MEASUREMENT_HeartRate',
  9: 'MEASUREMENT_Height',
  10: 'MEASUREMENT_LAB',
  11: 'MEASUREMENT_PainScale',
  12: 'MEASUREMENT_QTCB',
  13: 'MEASUREMENT_Res_Device',
  14: 'MEASUREMENT_Res_ETCO2',
  15: 'MEASUREMENT_Res_FIO2',
  16: 'MEASUREMENT_Res_GCS',
  17: 'MEASUREMENT_Res_O2',
  18: 'MEASUREMENT_Res_PEEP',
  19: 'MEASUREMENT_Res_PIP',
  20: 'MEASUREMENT_Res_RESP',
  21: 'MEASUREMENT_Res_SPO2',
  22: 'MEASUREMENT_Res_Tidal',
  23: 'MEASUREMENT_Res_Vent',
  24: 'MEASUREMENT_Rothman',
  25: 'MEASUREMENT_SOFA',
  26: 'MEASUREMENT_Temp',
  27: 'MEASUREMENT_Weight',
  28: 'OBSERVATION_ICU',
  29: 'OBSERVATION_LDA',
  30: 'OBSERVATION_Vent',
  31: 'OBSERVATION_Payer',
  32: 'OBSERVATION_Smoking',
  33: 'OBSERVATION_Zipcode',
  34: 'VISIT',
  35: 'query_log',
  36: 'location_v6'},
 'Count': {0: 226199,
  1: 5104,
  2: 228123

In [31]:
stage_t_dict['Count'][7]

19669584

In [26]:
def find_key(input_dict, value):
    return next((k for k, v in input_dict.items() if v == value), None)

7

In [41]:
loaded = omop.config_param['load']
result = []
for t in loaded.keys():
    if t not in ['provider','care_site','location']:
        if loaded[t]:
            for part in loaded[t]:
                stg_name = STAGE[t][part]
                idx = find_key(stage_t_dict['Table'], stg_name)
                count = stage_t_dict['Count'][idx]
                result.append([t, part, count])
        else:
            stg_name = STAGE[t]
            idx = find_key(stage_t_dict['Table'], stg_name)
            count = stage_t_dict['Count'][idx]
            result.append([t, part, count])

table_counts = pd.DataFrame(result, columns=['Table', 'Part', 'Count'])
table_counts.head()

,Table,Part,Count
0,person,zipcode,226199
1,death,zipcode,5104
2,condition_occurrence,zipcode,22812356
3,procedure_occurrence,icd,160254
4,procedure_occurrence,cpt,9888636


In [42]:
table_counts.groupby('Table').sum()

,Count
Table,
condition_occurrence,22812356
death,5104
drug_exposure,32171292
measurement,253833152
observation,30624205
person,226199
procedure_occurrence,10048890
visit,8798468
